In [118]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.linear_model import Lasso, Ridge

In [119]:
data = pd.read_csv('Advertising.csv')

In [120]:
data = data.drop('Unnamed: 0',axis=1)

In [121]:
new_data = data[:40]

In [155]:
new_data.shape

(40, 4)

In [209]:
X = new_data.drop('Sales', axis=1)
y = new_data['Sales']

## 1. Use LinearRegression to predict the sales using cross-validation, interpret the results why cross_val score for testing is increasing/ decreasing

In [210]:
from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test = train_test_split(X, y)

In [215]:
#Scaling the data
scale = StandardScaler()
sc = scale.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)


In [128]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [217]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = np.mean(cross_val_score(linreg, X_train, y_train, cv=5, scoring="neg_mean_squared_error"))
cv_5_results

-3.121539431218783

In [218]:
cv_5_results = np.mean(cross_val_score(linreg, X_test, y_test, cv=5, scoring="neg_mean_squared_error"))
cv_5_results

-11.328598375446953

In [129]:
y_train_pred = linreg.predict(X_train)
y_test_pred = linreg.predict(X_test)

In [178]:
train_score = linreg.score(X_train, y_train)
test_score = linreg.score(X_test, y_test)

print('Trainingr2:',train_score )
print('Testingr2:',test_score )

Trainingr2: 0.8696375446203377
Testingr2: 0.8777492233281081


##  2. Is it a Bias or Variance problem ? 

## 3. What method would you use to solve overfitting / underfitting problem?

In [170]:
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_poly_train = pd.DataFrame(poly.fit_transform(X_train), columns=poly.get_feature_names(X.columns))
X_poly_test = pd.DataFrame(poly.transform(X_test), columns=poly.get_feature_names(X.columns))
X_poly_train.head()

,TV,Radio,Newspaper,TV^2,TV Radio,TV Newspaper,Radio^2,Radio Newspaper,Newspaper^2
0,-0.373697,-0.439923,0.173525,0.139650,0.164398,-0.064846,0.193532,-0.076338,0.030111
1,0.892069,-0.473776,-0.337874,0.795788,-0.422641,-0.301407,0.224464,0.160077,0.114159
2,-0.545903,-1.516455,-0.181155,0.298010,0.827838,0.098893,2.299637,0.274714,0.032817
3,1.271580,-1.381043,-0.614195,1.616915,-1.756106,-0.780998,1.907278,0.848229,0.377235
4,0.049687,1.178262,0.994239,0.002469,0.058545,0.049401,1.388301,1.171474,0.988511


In [171]:
lr_poly = LinearRegression()
lr_poly.fit(X_poly_train,y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [179]:
train_poly_score = lr_poly.score(X_poly_train, y_train)
test_poly_score = lr_poly.score(X_poly_test, y_test)

print('Training r2:', train_poly_score)
print('Testing r2:', test_poly_score)

Training r2: 0.9896179697766543
Testing r2: 0.9884186864149485


In [175]:
b = bias(y_train, lr_poly.predict(X_poly_train)) #Your code here
v = variance(lr_poly.predict(X_poly_train)) #Your code here
print('Bias={} and Variance={}'.format(b,v))

Bias=9.25185853854297e-18 and Variance=0.8542490208259486


In [176]:
b = bias(y_test, lr_poly.predict(X_poly_test)) #Your code here
v = variance(lr_poly.predict(X_poly_test)) #Your code here
print('Bias={} and Variance={}'.format(b,v))

Bias=0.0212484900587654 and Variance=1.0385882507119764


## Feature Selection

In [168]:
bestfeatures = SelectKBest(score_func=chi2, k=9)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features

ValueError: k should be >=0, <= n_features = 3; got 9. Use k='all' to return all features.

## 4. Use Regularization, Which is better Lasso / Ridge / Simple Linear Regression

In [182]:
lasso = Lasso(alpha=.05) 
lasso.fit(X_train, y_train)
print('Training r2:', lasso.score(X_train, y_train))
print('Testing r2:', lasso.score(X_test, y_test))

Training r2: 0.8552719299169915
Testing r2: 0.8207648921282686


In [181]:
ridge = Ridge() #Lasso is also known as the L1 norm.
ridge.fit(X_train, y_train)
print('Training r2:', ridge.score(X_train, y_train))
print('Testing r2:', ridge.score(X_test, y_test))

Training r2: 0.8682322326931468
Testing r2: 0.8627751872621978
